# tensorboard 사용하기

실습환경

- OS: Ubuntu 14.04.3
- anaconda ver: 2.5.0
- ipython ver: 4.0.3
- python ver: 2.7.11
- tensorflow ver: 0.7.1


참조 강의

- 김성훈교수님 [lab9-2: Tensor Board로 딥네트웍 들여다보기](https://youtu.be/eDKxY5Z5dVQ)
- [구글 tensorflow 사이트](https://www.tensorflow.org/versions/r0.8/how_tos/summaries_and_tensorboard/index.html)


## 기술노트: 

### 1. tensorboard 실행 오류

오류 해결 참조문서: http://stackoverflow.com/questions/36615004/tensorflow-visualizer-tensorboard-not-working-under-anaconda

아나콘다에서 tensorboard 실행하면 다음과 같은 오류가 발생한다.

```
# tensorboard --logdir=./logs/test
Traceback (most recent call last):
  File "/root/anaconda2/bin/tensorboard", line 4, in <module>
    import tensorflow.tensorboard.tensorboard
ImportError: No module named tensorboard
```

이것은 anaconda에서 tensorboard.py가 backend/ 디렉토리 밑에 설치되기 때문에 발생하는 오류이다.

다음과 같이 실행하면 오류가 나지 않는다.

```
# python /root/anaconda2/lib/python2.7/site-packages/tensorflow/tensorboard/backend/tensorboard.py --logdir=./logs/test
```
 
또는 tensorboard.py를 tensorflow/tensorboard/ 디렉토리에 복사한다.

```
# cp /root/anaconda2/lib/python2.7/site-packages/tensorflow/tensorboard/backend/tensorboard.py /root/anaconda2/lib/python2.7/site-packages/tensorflow/tensorboard/
# tensorboard --logdir=.logs/test 
```

브라우저에서 실행 후 tensorboard 내용을 확인한다.

http://[서버ip]:6006/


### 2. tf.merge_all_summaires() 사용에 대해서

아래와 같은 오류가 발생할 수 있다.
```
merged = tf.merge_all_summaries()

InvalidArgumentError: You must feed a value for placeholder tensor 'Placeholder' with dtype float
	 [[Node: Placeholder = Placeholder[dtype=DT_FLOAT, shape=[], _device="/job:localhost/replica:0/task:0/cpu:0"]()]]
```

http://stackoverflow.com/questions/35413618/tensorflow-placeholder-error-when-using-tf-merge-all-summaries

를 참조해서 tf.merge_summary() 함수를 사용하도록 한다.

### 3. 기타 주의 사항

- safari ver 6.1.6 에서 tensorboard의 내용은 제대로 보여지지 않는다. 
- chrome 49.0.x 이상에서는 정상적으로 보여진다.





In [6]:
%pylab inline

import tensorflow as tf
import numpy as np

# 파라미터 선언
X = tf.placeholder(tf.float32)
Y = tf.placeholder(tf.float32)

add = tf.add(X, Y)
mul = tf.mul(X, Y)

# ----------------------------------------
# step 1: 주석을 달고 싶은 node를 선택한다.
# scalar_summary, histogram_summary 함수가 있다.
add_hist = tf.scalar_summary("add_scalar", add)
mul_hist = tf.scalar_summary("mul_scalar", mul)


# ----------------------------------------
# step 2: 모든 summary를 합친다.
#tf.reset_default_graph() => 사용하면 안된다.

# 아래 사용은 오류가 발생할 수 있다.
#merged = tf.merge_all_summaries()

# OK
merged = tf.merge_summary([add_hist, mul_hist])


with tf.Session() as sess:
    # 변수 초기화
    init = tf.initialize_all_variables()
    
    # 그래프 시작
    sess.run(init)

    # ----------------------------------------
    # step 3: writer를 생성한다.
    writer = tf.train.SummaryWriter("./logs/test2", sess.graph_def)

    for step in xrange(100):
        sess.run(add, feed_dict={X: step * 1.0, Y: 2.0})
        
        # -------------------------------------------------
        # step 4: summary merge와 add_summary()를 실행한다.
        summary = sess.run(merged, feed_dict={X: step * 1.0, Y: 2.0})
        writer.add_summary(summary, step)

print "done!"

Populating the interactive namespace from numpy and matplotlib
done!


`%matplotlib` prevents importing * from pylab and numpy
